# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Ana Paula Aravena Cifuentes  <br>
Url: https://github.com/rainamelui/03MIAR---Algoritmos-de-Optimizacion--2024<br>
Problema:
> 1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran toda la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

* Número de actores: 10
* Número de tomas : 30
* Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario

(*) La respuesta es obligatoria





                                        

#### (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.





Sin tener en cuenta restricciones, el número de posibilidades se define considerando que cada toma puede o no tener a cada uno de los 10 actores. Por esto el numero de posibilidades sería 2^(n_actores*n_tomas).

In [1]:
n_actores=10
n_tomas=30
2^(n_actores*n_tomas)

302

#### Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)



La estructura de datos que mejor se adapta al problema es un DataFrame de pandas, porque proporciona flexibilidad para manipular y analizar los datos.
En un principio habia esbozado las ideas utilizando una lista, pero debido a sus pocas funcionalidades integradas (que en este caso harian necesarias más bucles y funciones) y la complejidad asociada a acceder a los datos, se optó por cambiar a un DataFrame.

Los DataFrame ofrecen muchas ventajas cuando manejamos grandes bases de datos, por lo que al desarrollarlo con esta estructura se asegura una buena dimensionabilidad. De la misma manera, se pudieron optimizar procesos debido a que incluye funciones de suma y contar valores. Por último, hace del codigo más legible y entendible.

In [2]:
# Ejemplos de cada estructura mencionada
#DataFrame de pandas
import pandas as pd
df = pd.DataFrame({'1': [1,1,0],
                   '2': [1,0,0],
                   '3': [1,1,1]})
print("DataFrame:")
print(df)
# Ejemplo de lista
lista = [[1,1,1],
         [1,0,1],
         [0,0,1]]
print("Lista:")
print(lista)

# Diferencias para acceder a los datos
print(df.loc[2, '1']) #imprimo si el actor 1 participa en la toma 3
print(lista[0][0]) #imprimo si el actor 1 participa en la primera toma

DataFrame:
   1  2  3
0  1  1  1
1  1  0  1
2  0  0  1
Lista:
[[1, 1, 1], [1, 0, 1], [0, 0, 1]]
0
1


In [3]:
df.iloc[2, 0]

0

#### Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

La función objetivo es minimizar el gasto por los servicios de los actores de doblajes.

Las variables que intervienen en el problema se describen

* df_base: matriz de tomas $(i)$ y actores $(j)$. Se indican que actores intervienen en cada toma. Por lo que el elemento ${ij}$ de df_base es de valor **1** cuando en la toma $i$ interviene el actor $j$, si no interviene, toma el valor **0**.

*  $P_{dj}$: Indica si el día $d$ el actor $j$ actua en alguna toma, utilizando 1 de ser verdadero y 0 de ser falso.

Por lo tanto, la función objetivo de minimizacion es:

$\text{costo_total} = \sum_{j}\sum_{D} P_{dj}$


#### Diseña un algoritmo para resolver el problema por fuerza bruta

#### Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

La complejidad sería exponencial, O(2^(n*m)), ya que se consideran todas las posibles combinaciones.

#### (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

El algoritmo combina un enfoque heurístico para enfrentar el problema. Considera conceptos como la distancia de Hamming para calcular la diferencia entre las tomas, haciendo que se ordenen por similitud. Esto simplifica el proceso de la asignacion de días.

Por otro lado, aprovecha las funcionalidades de pandas para manipular los datos, mejorando la eficiencia de otras implementaciones mas rudimentarias.Además, para el cálculo considera las restricciones, lo que limita el universo de respuestas que tiene que calcular en comparación a uno de fuerza bruta.

In [4]:
import pandas as pd
import numpy as np

In [5]:
ruta_excel ="Datos problema doblaje(30 tomas, 10 actores).xlsx"
data = pd.read_excel(ruta_excel, index_col=0,skiprows=1)
db_base = data.iloc[:30, :10].copy()

In [6]:
def hamming(row1, row2):
  return sum(x != y for x, y in zip(row1, row2))
def orden_hamm(df, reference_row):
  distances = df.apply(lambda row: hamming(reference_row, row), axis=1)
  sorted_df = df.iloc[distances.argsort()]
  return sorted_df
def acum(db,j):
  df = db.iloc[(j-6):j,:len(db.columns)-1].copy()
  cumulative_df = pd.DataFrame()
  for i in range(len(df.columns)):
    selected_row = df.iloc[:,i]
    cumulative_sum = selected_row.cumsum()
    cumulative_df = pd.concat([cumulative_df, cumulative_sum], axis=1)
  cumulative_df[cumulative_df > 1] = 1
  cumulative_df = cumulative_df.fillna(0)
  cumulative_df["costo_actores"] = cumulative_df.sum(axis=1)
  cumulative_df["escenas"] = range(1, len(cumulative_df) + 1)
  cumulative_df["proporcion"] =  cumulative_df["costo_actores"]/cumulative_df["escenas"]
  df["costo_actores"] = cumulative_df["costo_actores"].tolist()
  df["escenas"] = cumulative_df["escenas"].tolist()
  df["proporcion"] = cumulative_df["proporcion"].tolist()
  filas_seleccionadas = df.loc[df["proporcion"] == min(df["proporcion"])]
  ultima_fila=filas_seleccionadas.iloc[-1].name
  mejor=df.loc[:int(filas_seleccionadas.iloc[-1].name),:]
  return mejor , ultima_fila
def asignar_dias(db):
  j = 5
  dia_asignado = 0
  grupos_escenas = pd.DataFrame({"Dia": np.nan}, index=range(len(db)))
  result_dfs = []
  go=True
  while go:
    mejor_grupo, ultima_fila = acum(db, j+1)
    dia_asignado += 1
    grupos_escenas = mejor_grupo
    grupos_escenas["Dia"] = dia_asignado
    result_dfs.append(grupos_escenas.copy())
    j = ultima_fila + 6
    if ultima_fila>=len(db)-1:
      final_result_df = pd.concat(result_dfs, ignore_index=True)
      go=False
      return final_result_df
def imprimir_tomas(db_base, tabla2):
  tabla1 = pd.DataFrame({'Concatenado': db_base.astype(str).iloc[:, :10].apply(lambda x: ''.join(x.str[0]), axis=1), 'Toma': db_base.index})
  tabla2['Concatenado'] = tabla2.astype(str).iloc[:, :10].apply(lambda x: ''.join(x.str[0]), axis=1)
  finallll = tabla2[["Concatenado", "Dia"]].merge(tabla1[["Concatenado", "Toma"]], how='outer', on='Concatenado').drop_duplicates()
  grupos_dia = finallll.groupby('Dia')
  for dia, grupo in grupos_dia:
    tomas = ', '.join(map(str, grupo['Toma']))
    print(f"Tomas del día {dia}: {tomas}")
  tabla2.drop(columns=['Concatenado'], inplace=True)

In [7]:
def optimizar_dias(df):
  sorted_df = pd.DataFrame(columns=df.columns)
  base=df.copy()
  for i in range(len(df)):
    reference_row = df.iloc[0]
    sorted_df = pd.concat([sorted_df, df.iloc[[0]]], ignore_index=True)
    df = orden_hamm(df.drop(df.index[0]), reference_row)
  sorted_df["Distancia"] = sorted_df.apply(lambda row: hamming(row, sorted_df.shift(1).iloc[row.name]), axis=1)
  sorted_df.loc[0, "Distancia"] = None
  resultado=asignar_dias(sorted_df)
  final=resultado.iloc[:, :len(df.columns)].copy()
  final["Dia"] = resultado["Dia"]
  costo_total=final.groupby('Dia').sum().clip(upper=1).sum().sum()
  print("El costo total es:",costo_total)
  tomas=imprimir_tomas(base,final)
  return final, costo_total

In [8]:
resultado=optimizar_dias(db_base)

El costo total es: 28.0
Tomas del día 1: 1, 6, 7, 9, 28, 30
Tomas del día 2: 13, 27, 2, 20, 22, 12
Tomas del día 3: 14, 17, 19, 23, 18, 24
Tomas del día 4: 21, 8, 10, 29, 26, 11
Tomas del día 5: 4, 15, 3, 5, 25, 16


C:\Users\Annie\AppData\Local\Temp\ipykernel_14008\3725193716.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sorted_df = pd.concat([sorted_df, df.iloc[[0]]], ignore_index=True)


#### (*)Calcula la complejidad del algoritmo

Respuesta

Para calcular la complejidad del algoritmo se observa la complejidad de cada una de las funciones definidas.


| Función                          | Complejidad                           | Justificación                                      |
|----------------------------------|---------------------------------------|-----------------------------------------------------|
| `hamming`               | O(m)      | Itera sobre cada elemento de las filas una vez.    |
| `orden_hamm`    | O(n * m * log(n))                 | Itera sobre cada fila y realiza una operación de ordenamiento (`argsort`) que tiene una complejidad de O(m * log(n)). |
| `acum`                           | O(n * m^2)                            | Itera sobre cada fila y realiza operaciones que toman tiempo cuadrático en la longitud de las filas. |
| `asignar_dias`                   | O(n * m^2)                           | Itera hasta que se alcanza el final del DataFrame. |
| `optimizar_dias`                 | O(m^2 * n * log(n))                   | Llama a `orden_hamm` y `asignar_dias`. Además, existen otras operaciones de complejidad similar o inferior dentro de la función. | |

Considerando:
*   m= Número de columnas
*   n= Número de filas

Así, la complejidad aproximada del algoritmo es **O(m^2 * n * log(n))**

#### Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios


In [9]:
def datos_aleatorios(num_actores, num_tomas):
  data = np.random.choice([0, 1], size=(num_actores,  num_tomas), p=[0.7, 0.3])
  columnas = [i for i in range(1,  num_tomas + 1)]
  filas = [i for i in range(1, num_actores + 1)]
  df = pd.DataFrame(data, index=filas, columns=columnas)
  df.index.name = "Tomas"
  return df.astype(float)

actores = 40
tomas = 40
db_aleatoria= datos_aleatorios(tomas,actores)

db_aleatoria


,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
Tomas,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
8,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


#### Aplica el algoritmo al juego de datos generado

In [10]:
resultado_aleatorio=optimizar_dias(db_aleatoria)

C:\Users\Annie\AppData\Local\Temp\ipykernel_14008\3725193716.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sorted_df = pd.concat([sorted_df, df.iloc[[0]]], ignore_index=True)


El costo total es: 222.0
Tomas del día 1: 1, 14, 38, 28, 27, 4
Tomas del día 2: 30, 24, 39, 20, 34, 32
Tomas del día 3: 22, 10, 7, 31, 16, 33
Tomas del día 4: 23, 8, 40, 19, 26, 25
Tomas del día 5: 21, 12, 29, 11, 13, 17
Tomas del día 6: 36, 2, 37, 6, 9, 3
Tomas del día 7: 5, 35, 18, 15


#### Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Algorithm to sort rows and cols by similarity. Stack Overflow. Recuperado el 4 de febrero de 2024, de https://stackoverflow.com/questions/36543960/algorithm-to-sort-rows-and-cols-by-similarity

Matplotlib documentation — Matplotlib 3.8.2 documentation. Matplotlib.org. Recuperado el 31 de enero de 2024, de https://matplotlib.org/stable/index.html

NumPy Documentation. Numpy.org. Recuperado el 31 de enero de 2024, de https://numpy.org/doc/

Pandas documentation — pandas 2.2.0 documentation. Pydata.org. Recuperado el 31 de enero de 2024, de https://pandas.pydata.org/docs/

Sort table rows/columns to minimize distance. Cross Validated. Recuperado el 4 de febrero de 2024, de https://stats.stackexchange.com/questions/218253/sort-table-rows-columns-to-minimize-distance

Python: Integrity sorting in two lists, GeeksforGeeks.
Recuperado el 31 de enero de 2024, de https://www.geeksforgeeks.org/python-integrity-sorting-in-two-lists/

#### Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

El algoritmo implementado es bastante manual, por lo que para mejorar la solución y optimizarla se podrían utilizar técnicas como algoritmos genéticos o algoritmos de búsqueda local, buscando mejorar su eficiencia buscando soluciones óptimas. Si bien esto no tiene mucha importancia cuando tratamos con una base de datos pequeña, como es este caso, cuando crece en tamaño si es relevante buscar la solución más optimizada.